In [1]:
import pandas as pd
import os
 
 
data = pd.read_csv('Postive imdb reviews.csv', encoding='utf-8')
with open('Postive imdb reviews.txt','a+', encoding='utf-8') as f:
    for line in data.values:
        f.write((str(line[1])+'\n'))


In [2]:
# import and download necessary packages
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

# start reading the text file
# establish a connection to the external text file 'epl.txt'
infile = open('Postive imdb reviews.txt', 'r') 

# read all lines in epl.txt
lines = infile.readlines()

# remove \n at the end of each line 
lines = [l.strip() for l in lines]

# remove empty lines
lines = [l for l in lines if l != ""]

# display the number of lines in the text file
# note: each line is treated as a sole document
len(lines)

[nltk_data] Downloading package punkt to /Users/kawanwong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1039

In [3]:
# import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# convert our text data to a DTM
tf_vectorizer = CountVectorizer(stop_words='english')
dtm_epl = tf_vectorizer.fit_transform(lines)

In [4]:
# convert our DTM to a numpy array and display it
dtm_epl.toarray()

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
# import necessary library
from sklearn.decomposition import LatentDirichletAllocation

# train a latent dirichlet allocation model with number of topics = 10
lda = LatentDirichletAllocation(n_components=10, random_state=0)

# fit the dtm into the lda object
lda.fit(dtm_epl)

LatentDirichletAllocation(random_state=0)

In [6]:
# get the topic-word(term) association for the LDA object
topic_word_matrix = lda.components_

# retrieve top n_top_words words for each topic
no_top_words = 10
feature_names = tf_vectorizer.get_feature_names()

# create a dataframe for displaying the results
rows = []

for topic_id, topic in enumerate(topic_word_matrix):
    row = ["Topic #" + str(topic_id) + ":"]
    row += [
        feature_names[i] + "*" + str(np.round(topic[i] / np.sum(topic), 4))
        for i in topic.argsort()[:-no_top_words - 1:-1]
    ]
    rows.append(row)

topic_word_df = pd.DataFrame(rows, columns=['Topic', 'Top 1 Word*Prob', 'Top 2 Word*Prob', \
                                            'Top 3 Word*Prob', 'Top 4 Word*Prob', 'Top 5 Word*Prob', 'Top 6 Word*Prob', 'Top 7 Word*Prob', 'Top 8 Word*Prob', 'Top 9 Word*Prob', 'Top 10 Word*Prob'])

topic_word_df

,Topic,Top 1 Word*Prob,Top 2 Word*Prob,Top 3 Word*Prob,Top 4 Word*Prob,Top 5 Word*Prob,Top 6 Word*Prob,Top 7 Word*Prob,Top 8 Word*Prob,Top 9 Word*Prob,Top 10 Word*Prob
0,Topic #0:,series*0.0222,great*0.0155,best*0.0134,characters*0.0129,acting*0.0124,like*0.0119,really*0.0101,actors*0.0098,story*0.0091,watch*0.0088
1,Topic #1:,series*0.0251,good*0.0245,watch*0.0131,really*0.0096,like*0.0096,korean*0.0088,games*0.0085,acting*0.0078,episode*0.007,story*0.007
2,Topic #2:,series*0.018,story*0.013,episode*0.0095,characters*0.0092,korean*0.0085,watch*0.0085,like*0.0078,actors*0.006,really*0.0059,definitely*0.0059
3,Topic #3:,series*0.0136,great*0.012,watch*0.0103,like*0.0095,season*0.0087,people*0.0082,netflix*0.0074,story*0.0071,really*0.0068,acting*0.0064
4,Topic #4:,acting*0.0114,series*0.0092,games*0.0083,episode*0.0077,characters*0.0075,like*0.0074,watch*0.0067,korean*0.0067,episodes*0.0063,ending*0.0056
5,Topic #5:,series*0.0213,watch*0.0157,good*0.013,korean*0.0109,really*0.01,great*0.0099,end*0.0076,episodes*0.0074,like*0.0068,amazing*0.0067
6,Topic #6:,watch*0.0111,games*0.0087,really*0.008,best*0.0068,dont*0.0066,great*0.0064,good*0.006,know*0.0059,life*0.0056,acting*0.0055
7,Topic #7:,series*0.0239,episode*0.0145,watch*0.0091,story*0.0087,enjoyed*0.0075,like*0.0067,great*0.0063,season*0.0057,games*0.0057,end*0.0056
8,Topic #8:,really*0.0115,great*0.0097,watch*0.0081,people*0.0073,character*0.0068,good*0.0062,like*0.0057,watching*0.0054,shows*0.0051,characters*0.005
9,Topic #9:,series*0.0174,people*0.0107,good*0.0084,really*0.0066,watch*0.0065,vips*0.0061,point*0.006,shows*0.0055,recommend*0.0053,great*0.0052


In [7]:
# document-topic matrix
doc_topic_matrix = lda.transform(dtm_epl)

# specify the number of topics we're interested in
no_top_topic = 2
rows = []

# process each document
for doc_id, doc in enumerate(doc_topic_matrix):
    row = ['# %d' % doc_id]
    # get the top n topics
    row += [
        'Topic' + str(i) + "*" + str(np.round(doc[i], 2))
        for i in doc.argsort()[:-no_top_topic - 1:-1]
    ]
    rows.append(row)

doc_topic_df = pd.DataFrame(
    rows, columns=['Document', 'Top1 Topic*Prob', 'Top2 Topic*Prob'])

doc_topic_df

,Document,Top1 Topic*Prob,Top2 Topic*Prob
0,# 0,Topic3*0.98,Topic1*0.0
1,# 1,Topic0*0.97,Topic1*0.0
2,# 2,Topic0*0.98,Topic1*0.0
3,# 3,Topic1*0.82,Topic0*0.02
4,# 4,Topic7*0.54,Topic5*0.44
...,...,...,...
1034,# 1034,Topic3*0.87,Topic7*0.01
1035,# 1035,Topic3*0.94,Topic0*0.01
1036,# 1036,Topic0*0.91,Topic1*0.01
1037,# 1037,Topic7*0.91,Topic5*0.01


In [8]:
# document-topic matrix
doc_topic_matrix = lda.transform(dtm_epl)

# specify the number of topics we're interested in
no_top_topic = 2
rows = []

# process each document
for doc_id, doc in enumerate(doc_topic_matrix):
    row = ['# %d' % doc_id]
    # get the top n topics
    row += [
        'Topic' + str(i)
        for i in doc.argsort()[:-no_top_topic - 1:-1]
    ]
    rows.append(row)

doc_topic_df = pd.DataFrame(
    rows, columns=['Document', 'Top1 Topic*Prob', 'Top2 Topic*Prob'])

doc_topic_df

,Document,Top1 Topic*Prob,Top2 Topic*Prob
0,# 0,Topic3,Topic1
1,# 1,Topic0,Topic1
2,# 2,Topic0,Topic1
3,# 3,Topic1,Topic0
4,# 4,Topic7,Topic5
...,...,...,...
1034,# 1034,Topic3,Topic7
1035,# 1035,Topic3,Topic0
1036,# 1036,Topic0,Topic1
1037,# 1037,Topic7,Topic5


In [9]:
import pyLDAvis
import pyLDAvis.sklearn

import pyLDAvis.gensim_models

data = pyLDAvis.sklearn.prepare(lda,dtm_epl,tf_vectorizer,mds='mmds')

In [10]:
pyLDAvis.display(data)

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
pyLDAvis.save_html(data, 'imdb pos review.html')

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
